# Задача 5. Паром.

Получите accuracy > 0.89 на тестовом датасете. Можно пользоваться любым классификатором **из sklearn**. Ансамблями пользоваться можно.

### XtreemeGradientBoosting и нейронные сети запрещены. 

In [1]:
!pip install jupyter_black
import jupyter_black
jupyter_black.load()

import time
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

# Random state
RS = 1337

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 re

In [2]:
df = pd.read_csv(
    "/kaggle/input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv",
    index_col="PassengerId",
)
df = df.reset_index(drop=True)

## 1. Первичный анализ

In [3]:
def get_data_info(data):
    display(data.head(40))
    display(data.describe(include="all"))
    data.info()


get_data_info(df)

,Country,Firstname,Lastname,Sex,Age,Category,Survived
0,Sweden,ARVID KALLE,AADLI,M,62,P,0
1,Estonia,LEA,AALISTE,F,22,C,0
2,Estonia,AIRI,AAVASTE,F,21,C,0
3,Sweden,JURI,AAVIK,M,53,C,0
4,Sweden,BRITTA ELISABET,AHLSTROM,F,55,P,0
5,Sweden,GERD INGA MAGNHILD,AHLSTROM,F,71,P,0
6,Sweden,HJALMAR,AHLSTROM,M,60,P,0
7,Estonia,PILLE,AHMAN,F,18,P,0
8,Estonia,TORMI,AINSALU,M,30,C,0
9,Sweden,ANNA MARIA,ALDRIN,F,63,P,0


,Country,Firstname,Lastname,Sex,Age,Category,Survived
count,989,989,989,989,989.000000,989,989.000000
unique,16,849,774,2,NaN,2,NaN
top,Sweden,ANDRES,ANDERSSON,M,NaN,P,NaN
freq,550,8,15,503,NaN,796,NaN
mean,NaN,NaN,NaN,NaN,44.575329,NaN,0.138524
std,NaN,NaN,NaN,NaN,17.235146,NaN,0.345624
min,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000
25%,NaN,NaN,NaN,NaN,30.000000,NaN,0.000000
50%,NaN,NaN,NaN,NaN,44.000000,NaN,0.000000
75%,NaN,NaN,NaN,NaN,59.000000,NaN,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    989 non-null    object
 1   Firstname  989 non-null    object
 2   Lastname   989 non-null    object
 3   Sex        989 non-null    object
 4   Age        989 non-null    int64 
 5   Category   989 non-null    object
 6   Survived   989 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 54.2+ KB


In [4]:
# Проверка на пропуски
pd.DataFrame(
    round(
        df.isna().mean() * 100,
    )
).style.background_gradient("coolwarm")

,0
Country,0.000000
Firstname,0.000000
Lastname,0.000000
Sex,0.000000
Age,0.000000
Category,0.000000
Survived,0.000000


### Вывод:
1. Категориальные признаки Country, Sex, Category необходимо закодировать
2. Колонку с возрастом можем сделать категориальной
3. Колонки с именем и фамилией не информативны, удалим их
4. Пропусков нет

## 2. Подготовка данных

### 2.1. Добавим категориальный столбец для возраста

In [5]:
def age_type(line):

    if not line:
        return "None"

    if line > 64:
        return "Old"

    if line > 30:
        return "Middle"

    if line > 17:
        return "Young"

    if line > 2:
        return "Child"

    return "Baby"


df["AgeType"] = df["Age"].apply(age_type)

### Большинство пассажиров было из Швеции и Эстонии, оставим эти страны, остальные объединим под одним типом

In [6]:
df["CountryType"] = df.Country.apply(
    lambda x: "Estonia" if x == "Estonia" else ("Sweden" if x == "Sweden" else "Others")
)

### 2.2. Обозначим признаки и цель прогнозирования

In [7]:
features = df.drop(["Survived", "Country", "Firstname", "Lastname"], axis=1)
target = df["Survived"]

### 2.3. Разобъем данные на обучающую и тестовую выборки

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    test_size=0.3,
    random_state=RS,
    stratify=df["Survived"],
)

### 2.4. Кодирование данных

In [9]:
numeric_features = ["Age"]
categorical_features = ["Sex", "Category", "AgeType", "CountryType"]

column_transformer = ColumnTransformer(
    transformers=[
        (
            "ohe",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            categorical_features,
        ),
        ("scaling", StandardScaler(), numeric_features),
    ],
    remainder="passthrough",
)

## 3. Обучение модели
Обучаем ансамбль моделей: knn, gradient boosting, linear regression, random forest. Используем баланс классов

In [10]:
knn = Pipeline(
    steps=[
        ("preprocessor", column_transformer),
        (
            "classifier",
            KNeighborsClassifier(),
        ),
    ]
)

gb = Pipeline(
    steps=[
        ("preprocessor", column_transformer),
        (
            "classifier",
            GradientBoostingClassifier(random_state=RS),
        ),
    ]
)

lr = Pipeline(
    steps=[
        ("preprocessor", column_transformer),
        (
            "classifier",
            LogisticRegression(random_state=RS, class_weight="balanced"),
        ),
    ]
)

rf = Pipeline(
    steps=[
        ("ohe_and_scaling", column_transformer),
        (
            "BalanceRF",
            RandomForestClassifier(
                class_weight={0: 0.423, 1: 0.589},
                random_state=RS,
                n_estimators=500,
                n_jobs=-1,
            ),
        ),
    ]
)

ensemble = VotingClassifier(
    estimators=[
        ("rf", rf),
        ("lr", lr),
        ("gb", gb),
        ("knn", knn),
    ],
    voting="soft",
)

## 4. Проверка на тестовой выборке

In [11]:
model = ensemble.fit(X_train, y_train)
y_preds = model.predict(X_test)
acc = accuracy_score(y_test, y_preds)

acc

0.8585858585858586